In [382]:
import numpy as np
import plotly as plt
from tqdm import tqdm

In [383]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px

We use the values of parameters from the paper, apart from sigma whcih was unfortunately not specified.

In [384]:
# Parameters

# General
t = 0
T = 300
np.random.seed(100)

# Price
b = 0
sigma = 0.1
P_0 = 45
dt = 1 / 1000   # time difference for Brownian motion simulation

# Spread
intensity = 1

# Cash and inventory
Y_0 = 0
X_0 = 0
delta = 0.005

# For Monte Carlo
l_const = 100 # constant size of trades for random and constant benchamrk srategies
M = 1000 # number of paths generated
n = 100 # number of divisions for grid in y

# Spread

The spread process is the combination of two stochastic processes. The first one is nonhomogenuous Possion process $(N_t)_t$ with intensity $\lambda(t)$, which determines the times when spread is afected by buy and sell orders. The second one, being the discrete-time stationary Markov chain governs the subsequent states of spread (here there are $m=6$ states) with probability transition matrix $\bold{P}$. The resultuing spread is a continuous time Markov chain: 
$$S_t = \hat{S}_{N_t}, t\geq 0 $$

In [385]:
def poisson_point_process_sim(intensity, T):
    '''
    Returns the arrival times and the number of arrivals.

    Parameters:
        intensity (float): The intensity of the process
        T (float): The timespan

    Returns:
        N, T_i ((int, np.array)): Tuple with realisations of the Poisson Process   
    '''
    N = np.random.poisson(intensity * T)    # number of changes of spread
    T_i = np.cumsum(np.random.exponential(1 / intensity, N))     # arrival times of changes of spread

    return (N, T_i)

In [386]:
def spread_sim(initial_state, spread_values, T, matrix, N): 
    '''
    Simulates the spread process.

    Parameters:
        initial_state (int): The iniitial value of spread (at time 0)
        spread_values (list): Possible values of the spread
        T (float): The timespan
        matrix (np.array): Probability transition matrix of the Markov chain
        N (int): Number of arrivals

    Returns:
        s (list): Realisation of the spread process   
    '''
    U = np.random.uniform(0, 1, 1000)  # uniform random variable determining the subsequent values of the Markov chain
    matrix_sum = np.cumsum(matrix, axis = 1)   # cumulative probabilities
    s = [initial_state]   # initial state is 0
    for i in range(1, N):
        potential_position = np.sum(U[i] > matrix_sum[s[i - 1], ])   # checking the positions based on the draw of the uniform random variable
        if potential_position == s[i - 1] or potential_position == 6:   # can't stay in the same state
            s.append(potential_position - 1)
        else:
            s.append(potential_position)
            
    return s

In [387]:
# spread specific objects

prob_matrix = np.array([[0, 0.41, 0.22, 0.16, 0.142, 0.065], 
                        [0.201, 0, 0.435, 0.192, 0.103, 0.067],
                        [0.113, 0.221, 0, 0.4582, 0.147, 0.059],
                        [0.07, 0.085, 0.275, 0, 0.465, 0.102],
                        [0.068, 0.049, 0.073, 0.363, 0, 0.446],
                        [0.077, 0.057, 0.059, 0.112, 0.692, 0]])   # probability transition matrix
                        
spread_values = [0.005, 0.01, 0.015, 0.02, 0.025, 0.03]  # posisible values of spread


In [388]:
PPP_spread = poisson_point_process_sim(intensity, T)   # realisation of tick time clock process - number and arrival times
spread_simulated = spread_sim(0, spread_values, T, prob_matrix, PPP_spread[0])   # simulated spread positions
s_t = np.array([spread_values[i] for i in spread_simulated])   # simulated spread values

In [389]:
fig = px.line(spread_simulated)
fig.update_traces(line_color = "maroon")

fig.update_layout(
    title_text="Spread evolution",
    title_x=0.5,
    plot_bgcolor='white',  # set background color to white
    paper_bgcolor='white',  # set background color to white
    font=dict(color='black'),  # set font color to black
    xaxis=dict(title_text="Index", color='black', tickcolor='black', gridcolor='black'),  # set x-axis color, tick color, grid color, and grid width
    yaxis=dict(title_text="Spread", color='black', tickcolor='black', gridcolor='black'),  # set y-axis color, tick color, grid color, and grid width
)
fig.show()

# Execution of orders

In [390]:
def orders_simulaton(intensity_matrix, spread_changes, spread, lambda_max, initial_intensities, q):
    '''
    Simulates the execution orders (either ask or bid) through acceptance-rejection algorithm.

    Parameters:
        intensity_matrix (np.array): Values of intensity in relation to market maker's choice of quotes and spread.
        spread_changes (np.array): Times when spread shifts
        spread (np.array): Values of spread
        lambda_max (float): Maximum possible value of lambda needed for acceptance-rejection algorithm, serving as a reference for accepting or rejecting the given arrival time
        initial_intensities (np.array): Arrival times for the orders but generated with lambda_max as intensity
        q (np.array): choice of quotes of the market maker

    Returns:
        (arrival_times, q_filtered) ((list, np.array)): Final arrival times of orders and matching quotes   
    '''
    lambda_process = []

    for t, i in zip(initial_intensities, range(len(initial_intensities) + 1)):
        spread_position = np.sum(t >= spread_changes)  # checking the spread at the arrival time t
        lambda_process.append(intensity_matrix[spread[spread_position - 1], q[i]])   # choosing the realisation of intensity process based on the estimated intensity function and simulated spread


    acceptance = np.where(lambda_process / lambda_max < np.random.uniform(0, 1, len(initial_intensities)), False, True)  # accepted arrival times

    arrival_times = initial_intensities[acceptance]  # final arrival times
    q_filtered = q[acceptance]  # filtered quotes - only at accepted arrival times

    return (arrival_times, q_filtered)

In [391]:
intensity_matrix_a = np.array([[0.0539, 0.1485], 
                              [0.0465, 0.0979],
                              [0.0401, 0.0846],
                              [0.0360, 0.0856],
                              [0.0435, 0.1009],
                              [0.0554, 0.1202]]) # Estimated intensities for given quotes and spread for ask

intensity_matrix_b = np.array([[0.0718, 0.1763], 
                              [0.0520, 0.1144],
                              [0.0419, 0.0915],
                              [0.0409, 0.0896],
                              [0.0452, 0.0930],
                              [0.0614, 0.1255]])    # Estimated intensities for given quotes and spread for bid

intensity_matrix_sym = (intensity_matrix_a + intensity_matrix_b) / 2    # Making bid and ask sides symmetric

In [392]:
lambda_max = np.max(intensity_matrix_sym)  # maximum lambda serving as a reference for acceptance probability
intensity_orders_a_simulation = poisson_point_process_sim(np.max(lambda_max), T) # simulation of the poisson process with maximum lambda as intensity for  ask
intensity_orders_b_simulation = poisson_point_process_sim(np.max(lambda_max), T) # simulation of the poisson process with maximum lambda as intensity for bid

q_a_initial = np.random.binomial(1, 0.5, len(intensity_orders_a_simulation[1]))  # ad-hoc choice of quotes for ask
q_b_initial = np.random.binomial(1, 0.5, len(intensity_orders_b_simulation[1]))  # ad-hoc choice of quotes for bid

In [393]:
orders_a_simulated = orders_simulaton(intensity_matrix_sym, PPP_spread[1], spread_simulated, lambda_max, intensity_orders_a_simulation[1], q_a_initial)
orders_b_simulated = orders_simulaton(intensity_matrix_sym, PPP_spread[1], spread_simulated, lambda_max, intensity_orders_b_simulation[1], q_b_initial)

orders_a = orders_a_simulated[0]  # final arrival times of ask orders
orders_b = orders_b_simulated[0]  # final arrival times of bid orders
q_a = orders_a_simulated[1]  # final quotes of ask orders at arrival times
q_b = orders_b_simulated[1]  # final quotes of bid orders at arrival times

In [394]:
import plotly.graph_objects as go

x_values = [0.005, 0.01, 0.015, 0.02, 0.025, 0.03]

fig = go.Figure()

# Add lines to the plot
fig.add_scatter(y=intensity_matrix_a[:,0], x=x_values, mode="lines", name="Ba", line_color="Maroon")
fig.add_scatter(y=intensity_matrix_a[:,1], x=x_values, mode="lines", name="Ba-", line_color="turquoise")
fig.add_scatter(y=intensity_matrix_b[:,0], x=x_values, mode="lines", name="Bb", line_color="darkorange")
fig.add_scatter(y=intensity_matrix_b[:,1], x=x_values, mode="lines", name="Bb+", line_color="green")

fig.update_layout(
    title_text="Plot of execution intensities as a function of the spread, expressed as frequency",
    title_x=0.5,
    plot_bgcolor='white',  # set background color to white
    paper_bgcolor='white',  # set background color to white
    font=dict(color='black'),  # set font color to black
    xaxis=dict(title_text="Tick", color='black', tickcolor='black', gridcolor='black'),  # set x-axis color, tick color, grid color, and grid width
    yaxis=dict(title_text="Execution intensities", color='black', tickcolor='black', gridcolor='black'),  # set y-axis color, tick color, grid color, and grid width
)

fig.show()

# Price process

The price is assumed to follow Bachelier model dynamics given by: 
$$dP_t=bdt+\sigma dW_t$$
Therefore:
$$P_t = P_0 + bt + \sigma W_t$$

In [395]:
def price_sim(b, P_0, t, sigma, W):
    return P_0 + b * t + sigma * W  # Bachelier model dynamics

In [396]:
timeline = np.sort(np.concatenate((orders_a, orders_b, PPP_spread[1])))  # merged times at which there is a trade execution or change of spread
W_t = np.cumsum(np.concatenate(([0], np.random.normal(0, np.sqrt(T * dt), int(1 / dt))))) # simulation of Brownian motion between subsequent times (variance equal to time difference)
t_price = np.arange(0, T + T * dt / 2, T * dt) 

p_t = price_sim(b, P_0, t_price, sigma, np.array(W_t))  # final price process

In [397]:
fig = px.line(p_t)
fig.update_traces(line_color = "maroon")

fig.update_layout(
    title_text="Price evolution",
    title_x=0.5,
    plot_bgcolor='white',  # set background color to white
    paper_bgcolor='white',  # set background color to white
    font=dict(color='black'),  # set font color to black
    xaxis=dict(title_text="Index", color='black', tickcolor='black', gridcolor='black'),  # set x-axis color, tick color, grid color, and grid width
    yaxis=dict(title_text="Price", color='black', tickcolor='black', gridcolor='black'),  # set y-axis color, tick color, grid color, and grid width
)

fig.show()

# Cash and inventory

The inventory follows the natural dynamics given by arrival times of buy and sell orders and sizes of the respective transactions ($L_t^a$ and $L_t^b$):

$$dY_t = L_t^bdN_t^b - L_t^adN_t^a$$

The cash dynamics is very similar with reversed signs (when the market makers buy, he/she uses cash and vice versa) and prices of the market maker $\pi^b(Q_t^b, P_{t^-}, S_{t^-})$ and $\pi^a(Q_t^a, P_{t^-}, S_{t^-})$:

$$dX_t = -\pi^b(Q_t^b, P_{t^-}, S_{t^-}) L_t^bdN_t^b + \pi^a(Q_t^a, P_{t^-}, S_{t^-}) L_t^adN_t^a$$

where 

$$\pi^b(Q_t^b, P_{t^-}, S_{t^-}) = 
\begin{cases}
    p-\frac{s}{2} & \text{for } q_b = Bb \\
    p-\frac{s}{2} + \delta & \text{for } q_b = Bb_+
\end{cases}$$

$$\pi^a(Q_t^a, P_{t^-}, S_{t^-}) = 
\begin{cases}
    p+\frac{s}{2} & \text{for } q_a = Ba \\
    p+\frac{s}{2} - \delta & \text{for } q_a = Ba_-
\end{cases}$$

where $p$ is the price, $s$ is the spread, $Bb$, $Ba$ are the choices of the existing spread and $Bb_+$, $Ba_-$ are the choices of spread updated by a tick to have a more favourable position.


In [398]:
def inventory_sim(N_a, N_b, l_a, l_b, Y_0):
    '''
    Simulates the inventory process.

    Parameters:
        N_a (np.array): Arrival times of buy orders
        N_b (np.array): Arrival times of sell orders
        l_a (np.array): Sizes of buy orders
        l_b (np.array): Sizes of sell orders
        Y_0 (float): Initial inventory (at time t=0)

    Returns:
       Y (np.array): Realisation of the inventory process
    '''
    Y = [Y_0]
    index_a = 0   # counter for sizes of ask orders 
    index_b = 0   # counter for sizes of bid orders 

    for t, index_y in zip(np.sort(np.concatenate((N_a, N_b))), range(1, len(np.concatenate((N_a, N_b)) + 1))):      # merged times of orders
        
        # check whether at a given time it is a bid or an ask order 
        if t in N_b:        
            Y.append(Y[index_y - 1] + l_b[index_b])   # if bid - add inventory
            index_b =+ 1
        else:
            Y.append(Y[index_y - 1] - l_a[index_a])   # if ask - subtract inventory
            index_a =+ 1

    return Y

In [399]:
def cash_sim(N_a, N_b, l_a, l_b, X_0, delta, q_a, q_b, s, spread_changes, p, price_changes):
    '''
    Simulates the cash process.

    Parameters:
        N_a (np.array): Arrival times of buy orders
        N_b (np.array): Arrival times of sell orders
        l_a (np.array): Sizes of buy orders
        l_b (np.array): Sizes of sell orders
        X_0 (float): Initial cash (at time t=0)
        delta (float): Parameter which may be used by a market maker to update the spread
        s (np.array): Realisation of the spread process
        spread_changes (np.array): Times when spread changes
        p (np.array): Realisation of the price process
        price_changes (np.array): Times when price changes

    Returns:
       X (np.array): Realisation of the cash process
    '''

    X = [X_0]
    index_a = 0
    index_b = 0

    for t, index_x in zip(np.sort(np.concatenate((N_a, N_b))), range(1, len(np.concatenate((N_a, N_b)) + 1))):
        
        spread_position = np.sum(t >= spread_changes)  # extracting spread at time t
        price_position = np.sum(t >= price_changes)  # extracting price at time t
        
        # check whether at a given time it is a bid or an ask order 
        if t in N_b:
            if q_b[index_b] == 1:   # checking which quote to use
                pi_b = p[price_position - 1] - s[spread_position - 1] / 2 + delta
            else:
                pi_b = p[price_position - 1] - s[spread_position - 1] / 2
            X.append((X[index_x - 1] - pi_b * l_b[index_b]))    # if bid subtract cash as I bought
            index_b =+ 1
        else:
            if q_a[index_a] == 1:
                pi_a = p[price_position - 1] + s[spread_position - 1] / 2 - delta
            else:
                pi_a = p[price_position - 1] + s[spread_position - 1] / 2
            X.append((X[index_x - 1] + pi_a * l_a[index_a]))  # If ask add cash as I sold
            index_a =+ 1

    return X

In [400]:
l_a = np.repeat(l_const, len(orders_a))
l_b = np.repeat(l_const, len(orders_b))


In [401]:
Y_t = inventory_sim(orders_a, orders_b, l_a, l_b, Y_0)  # realisation of inventory
X_t = cash_sim(orders_a, orders_b, l_a, l_b, X_0, delta, q_a, q_b, s_t, PPP_spread[1], p_t, t_price) # realisation of cash


In [402]:
fig = px.line(Y_t)
fig.update_traces(line_color = "maroon")

fig.update_layout(
    title_text="Inventory evolution",
    title_x=0.5,
    plot_bgcolor='white',  # set background color to white
    paper_bgcolor='white',  # set background color to white
    font=dict(color='black'),  # set font color to black
    xaxis=dict(title_text="Index", color='black', tickcolor='black', gridcolor='black'),  # set x-axis color, tick color, grid color, and grid width
    yaxis=dict(title_text="Y", color='black', tickcolor='black', gridcolor='black'),  # set y-axis color, tick color, grid color, and grid width
)

fig.show()

In [403]:
fig = px.line(X_t)
fig.update_traces(line_color = "maroon")
fig.update_layout(
    title_text="Cash evolution",
    title_x=0.5,
    plot_bgcolor='white',  # set background color to white
    paper_bgcolor='white',  # set background color to white
    font=dict(color='black'),  # set font color to black
    xaxis=dict(title_text="Index", color='black', tickcolor='black', gridcolor='black'),  # set x-axis color, tick color, grid color, and grid width
    yaxis=dict(title_text="X", color='black', tickcolor='black', gridcolor='black'),  # set y-axis color, tick color, grid color, and grid width
)

fig.show()

# Implementation of strategies

In [418]:
def Monte_Carlo_path(intensity_matrix, T, dt, X_0, Y_0, P_0, b, sigma, delta, l_const, initial_state_spread, spread_values, transition_matrix, intensity_spread, strategy):
    '''
    Generates the path for Monte Carlo simulation.

    Parameters:
        intensity_matrix (np.array): matrix with intensities depending on the spread position and choice of quote
        T (float): time span (here 300 s)
        dt (float): time difference in Euler scheme
        X_0 (float): Initial cash (at time t=0)
        Y_0 (float): Initial inventory (at time t=0)
        P_0 (float): Initial price (at time t=0)
        b (float):Drift parameter for price process
        sigma (float): Parameter which may be used by a market maker to update the spread
        delta (float): Volatility of the price
        l_const (float): The constant value for sizes of orders
        initial_state_spread (np.array): Realisation of the spread process
        spread_values (int): Initial value of spread (1 to 6)
        transition_matrix (np.array): Probability transition matrix for Markov chain generating the spread
        intensity_spread (np.array): Intensity for the spread process
        strategy (str): strategy choice, can be "constant" or "random"

    Returns:
       Y_t[-1], X_t[-1] / 100, N_a, N_b, max(Y_t) (np.array, np.array, np.array, np.array, np.array): Realisation of the terminal cash, inventory process, the number of executed bid and ask orders and maximum inventory
    '''
    # spread

    PPP_spread = poisson_point_process_sim(intensity_spread, T)   # realisation of tick time clock process - number and arrival times
    spread_simulated = spread_sim(0, spread_values, T, transition_matrix, PPP_spread[0])   # simulated spread positions
    s_t = np.array([spread_values[i] for i in spread_simulated])   # simulated spread values

    # execution of orders

    lambda_max = np.max(intensity_matrix) 
    intensity_orders_a_simulation = poisson_point_process_sim(np.max(lambda_max), T) # simulation of the poisson process with maximum lambda as intensity
    intensity_orders_b_simulation = poisson_point_process_sim(np.max(lambda_max), T) # simulation of the poisson process with maximum lambda as intensity

    if strategy == "random":
        q_a = np.random.binomial(1, 0.5, len(intensity_orders_a_simulation[1]))  # choice of quotes for ask
        q_b = np.random.binomial(1, 0.5, len(intensity_orders_b_simulation[1]))  # choice of quotes for bid
    elif strategy == "constant":
        q_a = np.repeat(0, len(intensity_orders_a_simulation[1]))  # choice of quotes for ask
        q_b = np.repeat(0, len(intensity_orders_b_simulation[1]))  # choice of quotes for bid

    orders_a_simulated = orders_simulaton(intensity_matrix, PPP_spread[1], spread_simulated, lambda_max, intensity_orders_a_simulation[1], q_a)
    orders_b_simulated = orders_simulaton(intensity_matrix, PPP_spread[1], spread_simulated, lambda_max, intensity_orders_b_simulation[1], q_b)

    orders_a = orders_a_simulated[0]  # final arrival times of ask orders
    orders_b = orders_b_simulated[0]  # final arrival times of bid orders

    q_a = orders_a_simulated[1]  # final quotes of ask orders at arrival times
    q_b = orders_b_simulated[1]  # final quotes of bid orders at arrival times

    # price

    W_t = np.cumsum(np.concatenate(([0], np.random.normal(0, np.sqrt(T * dt), int(1 / dt))))) # simulation of Brownian motion between subsequent times (variance equal to time difference)
    t_price = np.arange(0, T + T * dt / 2, T * dt) 

    p_t = price_sim(b, P_0, t_price, sigma, np.array(W_t))  # final price process

    # the choice of sizes of trades

    l_a = np.repeat(l_const, len(orders_a))
    l_b = np.repeat(l_const, len(orders_b))

    # inventory and cash
    
    Y_t = inventory_sim(orders_a, orders_b, l_a, l_b, Y_0)  # realisation of inventory
    X_t = cash_sim(orders_a, orders_b, l_a, l_b, X_0, delta, q_a, q_b, s_t, PPP_spread[1], p_t, t_price) # realisation of cash

    return Y_t[-1], X_t[-1] / 100, len(orders_a), len(orders_b), max(Y_t)

## Random Strategy

In [419]:
n = 1000 # number of paths

arr_res_Y_rand = [None] * n
arr_res_X_rand = [None] * n
arr_res_or_a_rand = [None] * n
arr_res_or_b_rand = [None] * n
arr_res_max_Y_rand = [None] * n


for i in tqdm(range(n)):
    res_acc = Monte_Carlo_path(intensity_matrix_sym, T, dt, X_0, Y_0, P_0, b, sigma, delta, l_const, 0, spread_values, prob_matrix, intensity, "random")
    
    arr_res_Y_rand[i] = res_acc[0]
    arr_res_X_rand[i] = res_acc[1]
    arr_res_or_a_rand[i] = res_acc[2]
    arr_res_or_b_rand[i] = res_acc[3]
    arr_res_max_Y_rand[i] = res_acc[4]

100%|██████████| 1000/1000 [00:04<00:00, 204.92it/s]


In [420]:
m_XT_rand = np.mean(arr_res_X_rand)
std_XT_rand = np.std(arr_res_X_rand)
m_sup_Y_rand = np.mean(arr_res_max_Y_rand)

m_sup_Y_rand

524.3

In [421]:
m_XT_rand / std_XT_rand

-0.025031116807933584

## Constant Strategy

In order to do the strategies simulation we have to do as follows for each of the 1000 paths: 
1) simulate values for price and spread. 
2) simulate orders_a, orders_b based on the choice of spread q_a, q_b
3) simulate l_a, l_b
4) multiply l times price, corrected by spread (and tick if q == 1)
5) calculate cash and inventory

In [422]:
n = 1000 # number of paths

arr_res_Y_const = [None] * n
arr_res_X_const = [None] * n
arr_res_or_a_const = [None] * n
arr_res_or_b_const = [None] * n
arr_res_max_Y_const = [None] * n


for i in tqdm(range(n)):
    res_acc = Monte_Carlo_path(intensity_matrix_sym, T, dt, X_0, Y_0, P_0, b, sigma, delta, l_const, 0, spread_values, prob_matrix, intensity, "constant")
    
    arr_res_Y_const[i] = res_acc[0]
    arr_res_X_const[i] = res_acc[1]
    arr_res_or_a_const[i] = res_acc[2]
    arr_res_or_b_const[i] = res_acc[3]
    arr_res_max_Y_const[i] = res_acc[4]

100%|██████████| 1000/1000 [00:04<00:00, 221.75it/s]


In [423]:
# Terminal wealth constant
m_XT_cst = np.mean(arr_res_X_const)
std_XT_cst = np.std(arr_res_X_const)
m_std_XT_cst = m_XT_cst/std_XT_cst

#Num. of exec. at bid constant
lengths_b = arr_res_or_b_const

m_NbT_cst = np.mean(lengths_b)
std_NbT_cst = np.std(lengths_b)

#Num. of exec. at ask constant
lengths_a = arr_res_or_a_const

m_NaT_cst = np.mean(lengths_a)
std_NaT_cst = np.std(lengths_a)

#Maximum Inventory constant
m_sup_Y_cst = np.mean(arr_res_max_Y_const)
std_sup_Y_cst = np.std(arr_res_max_Y_const)


In [424]:
m_XT_cst / std_XT_cst

-0.015662297595742794

## Optimal strategy without market orders (WOMO)

In [474]:
dt = 100
y = np.linspace(-1000, 1000, 100) # mesh grid in inventory
timespan = [t for t in range(0, int(T / dt))]

In [516]:
def sim_womo(l_a, l_b, X_0, Y_0, delta, q_a, q_b, s, spread_positions, spread_changes, p, price_changes, y, timespan, initial_intensities, intensity_matrix, lambda_max):
    '''
    Simulates the cash, inventory and executions of orders using simialr methods as previous function, but also optimal trading strategy.

    Parameters:
        l_a (np.array): optimal sizes of buy orders
        l_b (np.array): optimal sizes of sell orders
        X_0 (float): Initial cash (at time t=0)
        Y_0 (float): Initial inventory (at time t=0)
        delta (float): Parameter which may be used by a market maker to update the spread
        s (np.array): Realisation of the spread process
        spread_positions (np.array): spread positions (1 to 6)
        spread_changes (np.array): Times when spread changes
        p (np.array): Realisation of the price process
        price_changes (np.array): Times when price changes
        y (np.array): vector of inventories
        timespan (np.array): times of evaluation of the Euler scheme
        initial_intensities (np.array): initial simulaton of orders as if they were generated with homogeneous Poisson Process
        intensity_matrix (np.array): matrix with intensities depending on the spread position and choice of quote
        lambda_max (np.array): maximum lambda used as the reference for acceptance-rejection scheme

    Returns:
       X, Y, number_a, number_b (np.array, np.array, np.array, np.array, np.array): Realisation of the cash, inventory process and the number of executed bid and ask orders
    '''

    X = [X_0]
    Y = [Y_0]
    number_a = 0  # number of buy orders
    number_b = 0  # number of sell orders
    index = 0

    for t in initial_intensities:
        
        spread_position = np.sum(t >= spread_changes)  # extracting spread at time t
        y_position = np.argmin(np.abs(y - Y[index - 1])) # extracting y
        t_position = np.argmin(np.abs(timespan - t)) # extracting the given time
        price_position = np.sum(t >= price_changes)  # extracting price at time t
        
        bid_or_ask = np.random.uniform(0, 1, 1)

        if bid_or_ask >= 0.5:    # equal chance of being an ask or bid trade
            lambda_t = intensity_matrix[spread_positions[spread_position - 1], int(q_b[t_position, spread_positions[spread_position - 1], y_position])]
        else:
            lambda_t = intensity_matrix[spread_positions[spread_position - 1], int(q_a[t_position, spread_positions[spread_position - 1], y_position])]

        if lambda_t / lambda_max < np.random.uniform(0, 1, 1):
    
        # check whether at a given time it is a bid or an ask order 
            if np.random.uniform(0, 1, 1) >= 0.5:   # equal chance it is a bid order or ask order

                if q_b[t_position, spread_positions[spread_position - 1], y_position] == 1:   # checking which quote to use
                    pi_b = p[price_position - 1] - s[spread_position - 1] / 2 + delta
                else:
                    pi_b = p[price_position - 1] - s[spread_position - 1] / 2
                X.append((X[index - 1] - pi_b * l_b[t_position, spread_positions[spread_position - 1], y_position]))    # if bid subtract cash as I bought
                Y.append(Y[index - 1] + l_b[t_position, spread_positions[spread_position - 1], y_position])  # if bid - add inventory
                number_a = number_a + 1

            else:
                if q_a[t_position, spread_positions[spread_position - 1], y_position] == 1:
                    pi_a = p[price_position - 1] + s[spread_position - 1] / 2 - delta
                else:
                    pi_a = p[price_position - 1] + s[spread_position - 1] / 2
                X.append((X[index - 1] + pi_a * l_a[t_position, spread_positions[spread_position - 1], y_position]))  # If ask add cash as I sold
                Y.append(Y[index - 1] - l_a[t_position, spread_positions[spread_position - 1], y_position]) # if ask - subtract inventory
                number_b = number_b + 1
            index = index + 1

    return X, Y, number_a, number_b

In [517]:
def Monte_Carlo_womo_path(q_a, q_b, l_a, l_b, intensity_matrix, T, dt, X_0, Y_0, P_0, b, sigma, delta, initial_state_spread, spread_values, transition_matrix, intensity_spread, y, timespan):
    '''
    Generates the path for Monte Carlo simulation.

    Parameters:
        q_a (np.array): Optimal choices of ask quotes
        q_b (np.array): Optimal choices of bid quotes
        l_a (np.array): Optimal sizes of buy orders
        l_b (np.array): Optimal sizes of sell orders
        intensity_matrix (np.array): matrix with intensities depending on the spread position and choice of quote
        T (float): time span (here 300 s)
        dt (float): time difference in Euler scheme
        X_0 (float): Initial cash (at time t=0)
        Y_0 (float): Initial inventory (at time t=0)
        P_0 (float): Initial price (at time t=0)
        b (float): Drift parameter in price process
        sigma (float): Parameter which may be used by a market maker to update the spread
        delta (float): Volatility of the price
        initial_state_spread (np.array): Realisation of the spread process
        spread_values (int): Initial value of spread (1 to 6)
        transition_matrix (np.array): Probability transition matrix for Markov chain generating the spread
        intensity_spread (np.array): Intensity for the spread process
        y (np.array): vector of inventorie
        timespan (np.array): times of evaluation of the Euler scheme

    Returns:
       Y_t[-1], X_t[-1] / 100, N_a, N_b, max(Y_t) (np.array, np.array, np.array, np.array, np.array): Realisation of the terminal cash, inventory process, the number of executed bid and ask orders and maximum inventory
    '''

    # spread

    PPP_spread = poisson_point_process_sim(intensity_spread, T)   # realisation of tick time clock process - number and arrival times
    spread_simulated = spread_sim(0, spread_values, T, transition_matrix, PPP_spread[0])   # simulated spread positions
    s_t = np.array([spread_values[i] for i in spread_simulated])   # simulated spread values

    # price

    W_t = np.cumsum(np.concatenate(([0], np.random.normal(0, np.sqrt(T * dt), int(1 / dt))))) # simulation of Brownian motion between subsequent times (variance equal to time difference)
    t_price = np.arange(0, T + T * dt / 2, T * dt) 

    p_t = price_sim(b, P_0, t_price, sigma, np.array(W_t))  # final price process

    # inventory and cash

    lambda_max = np.max(intensity_matrix)
    intensity_orders_womo = poisson_point_process_sim(np.max(lambda_max), T)
    WOMO_result = sim_womo(l_a, l_b, X_0, Y_0, delta, q_a, q_b, s_t, spread_simulated, PPP_spread[1], p_t, t_price, y, timespan, intensity_orders_womo[1], 2 * intensity_matrix, lambda_max)
    X_t = WOMO_result[0]
    Y_t = WOMO_result[1]
    N_a = WOMO_result[2]
    N_b = WOMO_result[3]


    return Y_t[-1], X_t[-1] / 100, N_a, N_b, max(Y_t)

In [518]:
q_b = np.load("q_b_solved.npy")[::-1, : , :]
l_b = np.load("l_b_solved.npy")[::-1, : , :]
q_a = np.load("q_a_solved.npy")[::-1, : , :]
l_a = np.load("l_a_solved.npy")[::-1, : , :]

In [519]:
n = 1000 # number of paths

arr_res_Y_womo = [None] * n
arr_res_X_womo = [None] * n
arr_res_or_a_womo = [None] * n
arr_res_or_b_womo = [None] * n
arr_res_max_Y_womo = [None] * n


for i in range(n):
    res_acc = Monte_Carlo_womo_path(q_a, q_b, l_a, l_b, intensity_matrix_sym, T, dt, X_0, Y_0, P_0, b, sigma, delta, 0, spread_values, prob_matrix, intensity, y, timespan)
    
    arr_res_Y_womo[i] = res_acc[0]
    arr_res_X_womo[i] = res_acc[1]
    arr_res_or_a_womo[i] = res_acc[2]
    arr_res_or_b_womo[i] = res_acc[3]
    arr_res_max_Y_womo[i] = res_acc[4]

In [521]:
# Terminal wealth constant
m_XT_womo = np.mean(arr_res_X_womo)
std_XT_womo = np.std(arr_res_X_womo)
m_std_XT_womo = m_XT_cst / std_XT_cst

m_NbT_womo = np.mean(arr_res_or_b_womo)
std_NbT_womo = np.std(arr_res_or_b_womo)

m_NaT_womo = np.mean(arr_res_or_a_womo)
std_NaT_womo = np.std(arr_res_or_a_womo)

#Maximum Inventory constant
m_sup_Y_cst = np.mean(arr_res_max_Y_womo)
std_sup_Y_cst = np.std(arr_res_max_Y_womo)


In [522]:
m_std_XT_womo

-0.015662297595742794

In [531]:
fig = px.histogram(arr_res_X_womo, nbins=75)

fig.update_layout(
    title_text="Performance empirical distribution of constant strategy",
    title_x=0.5,
    plot_bgcolor='white',  # set background color to white
    paper_bgcolor='white',  # set background color to white
    font=dict(color='black'),  # set font color to black
    xaxis=dict(title_text="Value", color='black', tickcolor='black', gridcolor='black'),  # set x-axis color, tick color, grid color, and grid width
    yaxis=dict(title_text="Number of occurances", color='black', tickcolor='black', gridcolor='black'),  # set y-axis color, tick color, grid color, and grid width
)

In [532]:
fig = px.histogram(arr_res_or_b_womo, nbins=75)

fig.update_layout(
    title_text="N bid empirical distribution, constant strategy",
    title_x=0.5,
    plot_bgcolor='white',  # set background color to white
    paper_bgcolor='white',  # set background color to white
    font=dict(color='black'),  # set font color to black
    xaxis=dict(title_text="Value", color='black', tickcolor='black', gridcolor='black'),  # set x-axis color, tick color, grid color, and grid width
    yaxis=dict(title_text="Number of occurances", color='black', tickcolor='black', gridcolor='black'),  # set y-axis color, tick color, grid color, and grid width
)

In [533]:
fig = px.histogram(arr_res_or_a_womo, nbins=75)

fig.update_layout(
    title_text="N ask empirical distribution, constant strategy",
    title_x=0.5,
    plot_bgcolor='white',  # set background color to white
    paper_bgcolor='white',  # set background color to white
    font=dict(color='black'),  # set font color to black
    xaxis=dict(title_text="Value", color='black', tickcolor='black', gridcolor='black'),  # set x-axis color, tick color, grid color, and grid width
    yaxis=dict(title_text="Number of occurances", color='black', tickcolor='black', gridcolor='black'),  # set y-axis color, tick color, grid color, and grid width
)

In [534]:
fig = px.histogram(arr_res_or_b_womo, nbins=75)

fig.update_layout(
    title_text="N bid empirical distribution, random strategy",
    title_x=0.5,
    plot_bgcolor='white',  # set background color to white
    paper_bgcolor='white',  # set background color to white
    font=dict(color='black'),  # set font color to black
    xaxis=dict(title_text="Value", color='black', tickcolor='black', gridcolor='black'),  # set x-axis color, tick color, grid color, and grid width
    yaxis=dict(title_text="Number of occurances", color='black', tickcolor='black', gridcolor='black'),  # set y-axis color, tick color, grid color, and grid width
)

In [535]:
fig = px.histogram(arr_res_or_a_womo, nbins=75)

fig.update_layout(
    title_text="N ask empirical distribution, random strategy",
    title_x=0.5,
    plot_bgcolor='white',  # set background color to white
    paper_bgcolor='white',  # set background color to white
    font=dict(color='black'),  # set font color to black
    xaxis=dict(title_text="Value", color='black', tickcolor='black', gridcolor='black'),  # set x-axis color, tick color, grid color, and grid width
    yaxis=dict(title_text="Number of occurances", color='black', tickcolor='black', gridcolor='black'),  # set y-axis color, tick color, grid color, and grid width
)